Load model and embedding data.

In [5]:
import theano
import theano.tensor as T
import numpy as np
import cPickle
import random
from utils import *
from state import *
from title_model import TitleModel

theano.config.floatX='float64'

model_name = 'model/title_emb256_h256_rmsprop_f64_model.npz'

state = title_state()
model = TitleModel(state, test_mode=True)
model.load(model_name)

(ind2word, word2ind, _, _, _) = cPickle.load(open('data/dict.pkl'))

print('Data loaded.')

Data loaded.


Now we try to mannually provide an input sentence (if a word is out of vocab, we simply remove it).

In [11]:
def restoreW(ind_lst, ind2word):
    res = []
    for ind in ind_lst:
        if ind in ind2word:
            res.append(ind2word[ind])
    return ' '.join(res)
    

test_sents = ['i would like to go hiking tomorrow']

for (k, test_sent) in enumerate(test_sents):
    print "Test sent:", test_sent
    words = test_sent.split()
    nat_coded = [1]
    tok_set = range(20)
    for w in words:
        if w in word2ind:
            nat_coded.append(word2ind[w])
        else:
            if w in tmp_map:
                nat_coded.append(tmp_map[w])
            else:
                tok_ind = random.choice(tok_set)
                tok_s = '<TOK%d>' % tok_ind
                tok_set.remove(tok_ind)
                nat_coded.append(word2ind[tok_s])
            print '  out of vocab: %s' % w
    nat_coded.append(0)
    print 'Coded input:', nat_coded
    print restoreW(nat_coded, ind2word)

    m = state['seq_len_in']
    nat_coded_mat = numpy.zeros((m, 2), dtype='int32')
    nat_mask = numpy.zeros((m, 2), dtype='float64')
    sent_len = len(nat_coded)
    nat_coded_mat[:sent_len, 0] = nat_coded
    nat_mask[:sent_len, 0] = 1
    nat_coded_mat[:sent_len, 1] = nat_coded
    nat_mask[:sent_len, 1] = 1
    pred_fn = model.build_gen_function()
    
    res = [1]
    abs_in = 1
    model.gen_reset()
    while True:
        abs_in_mat = np.zeros((2, ), dtype='int32') + abs_in
        #print 'abs_in', abs_in_mat
        [p_t, o_t, alpha_t] = pred_fn(nat_coded_mat, nat_mask, abs_in_mat)
        #print "ot", o_t, ind2word[o_t[0]]
        pt_col = p_t[0]
        alpha_t = alpha_t[:, 0]
        #print alpha_t
        alpha_s = alpha_t.argsort()[::-1]
        #print sum(pt_col)
        pt_norm = [1.0 * a / sum(pt_col) for a in pt_col]
        #print pt_norm
        ind = np.asarray(pt_norm).argmax()
        abs_in = ind
        res.append(ind)
        if ind == 0 or len(res) > 10:        
            break
        
        print 'Explanation of: %s' % restoreW([ind], ind2word)
        for k in alpha_s[:len(nat_coded)]:
            print "    %s: %.4f" % (restoreW([nat_coded[k]], ind2word), alpha_t[k])
    print
    print restoreW(res, ind2word)
    print

Test sent: i would like to go hiking tomorrow
  out of vocab: hiking
  out of vocab: tomorrow
Coded input: [1, 1014, 109, 1051, 41, 603, 17, 18, 0]
<START> i would like to go <TOK15> <TOK16> <END>
Explanation of: <NO>
    <END>: 0.2341
    i: 0.1823
    <TOK16>: 0.1695
    <START>: 0.1472
    would: 0.1222
    <TOK15>: 0.0513
    like: 0.0448
    to: 0.0249
    go: 0.0236

<START> <NO> <END>

